### Homework 1

pubmed_cleaned.zip is a file containing a metadatas sample from pubmed articles. Your goal is to **convert the json to a mongo DB** and answer the following questions **USING** mongodb querys:

1) Create an index, explain your choice of key.

2) Delete every paper that was published prior 2019

3) How many paper have a single author ? Two authors ?

4) What's the last paper inserted in the db ?

5) Find articles with null meshwords.

6) Choose a keyword you are interested in (machine learning, computer vision,...). Find the number of articles with the choosen keyword in their meshwords, abstract or title.

7) What's the number of articles that have atleast one affiliation AND meshwords.

8) How many articles have a publishing date after 2020 ?

9) Find articles where there's atleast one affiliation from a choosen country (you decide which one).

10) Check for any duplicates. (hint: look at the doi or the pmid)

11) Remove every articles where the abstract starts with an "R".

12) Return the list of papers (pmid) where there's atleast one affiliation per author

13) Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples

14) Sandbox exercise: think of a problematic and try to answer it.

### Homework 2

authors.zip is a file containing a sample of authors that wrote a paper published on pubmed. Each doc as, at most, 5 keys. "AND_ID" is the disambiguated author id. "pmid_list" is the list of ids that the author published. "more_info" is a list of dict with each dict representing info for a given paper. "oa04_affiliations" is a list of dict with each dict representing affiliation info for a given paper. "oa06_researcher_education" is a list of dict with each dict containing information on the education of the researcher.

Your goal is to **convert the json to a mongo DB** and answer the following questions **USING** mongodb querys:

1) Create an index, explain your choice of key.

2) What is the average length of "pmid_list"

3) How many distinct affiliations are there ?

4) Find authors with atleast one "COM" AffiliationType

5) How many authors switched the AffiliationType ?

6) Find affiliation with the word "China" 

7) Get the pmids of papers published in 2019

8) Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and with the "oa06_researcher_education" AND "oa04_affiliations" .

9) What's the average "BeginYear" of "oa06_researcher_education".

10) Count the distinct country of "oa06_researcher_education"

11) Does the length of pmid_list and more_info always match ?

12) Does the length of pmid_list and "oa04_affiliations" always match ?

13) Sandbox exercise: think of a problematic and try to answer it.